In [8]:
import os
from dotenv import load_dotenv

# .envファイルを読み込む
load_dotenv()

True

In [21]:
import json
import feedparser
import openai
from openai import OpenAI
import requests
import logging
import os
from datetime import datetime, timedelta

logger = logging.getLogger()
logger.setLevel(logging.INFO)

processed_news_file = "processed_news.json"


def load_processed_news():
    if os.path.exists(processed_news_file):
        with open(processed_news_file, "r") as file:
            return json.load(file)
    return {}


def save_processed_news(processed_news):
    with open(processed_news_file, "w") as file:
        json.dump(processed_news, file)


def clean_old_news(processed_news, days=7):
    threshold_date = datetime.now() - timedelta(days=days)
    threshold_timestamp = int(threshold_date.timestamp())
    return {k: v for k, v in processed_news.items() if v > threshold_timestamp}


def get_rss_feeds(urls):
    all_entries = []
    for url in urls:
        try:
            feed = feedparser.parse(url)
            if feed.bozo:
                logger.error(f"Failed to parse feed: {url}")
            else:
                all_entries.extend(feed.entries)
        except Exception as e:
            logger.error(f"Error fetching feed {url}: {e}")
    return all_entries


def filter_ai_news(feed_entries):
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        logger.error("OpenAI API key is not set.")
        return []

    openai.api_key = openai_api_key
    client = OpenAI()

    filtered_entries = []

    for entry in feed_entries:
        title = entry.title
        description = entry.get("description", "")
        content = entry.get("content", "")[:300]

        prompt = f"""
与えられた文章が、以下の条件に合致する場合は1、そうでない場合は0を出力せよ。結果は0か1のみを出力すること。
# 条件
[LLM, 生成AI, 生成系AI, 基盤モデル, 大規模言語モデル, ChatGPT, OpenAI, Gemini, Claude, RAG]のいずれかに関連すること。
# 文章
{title}
{description}
{content}
# 結果
result=
"""
        logger.debug(f"filter_prompt: {prompt}")

        try:
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo", messages=[{"role": "system", "content": prompt}]
            )
            ai_decision = completion.choices[0].message.content.strip()
            if ai_decision == "1":
                filtered_entries.append(entry)
        except Exception as e:
            logger.error(f"Error in AI filtering: {e}")

    return filtered_entries


def is_news_processed(news_id, processed_news):
    return news_id in processed_news


def mark_news_as_processed(news_id, processed_news):
    processed_news[news_id] = int(datetime.now().timestamp())


def summarize_news(news_entries, processed_news):
    summaries = []
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        logger.error("OpenAI API key is not set.")
        return summaries

    openai.api_key = openai_api_key
    client = OpenAI()

    for entry in news_entries:
        news_id = entry.get("link")
        if is_news_processed(news_id, processed_news):
            logger.info(f"News already processed: {entry.title}")
            continue
        try:
            prompt = f"""
以下の記事のContentをFormatに従って要約して下さい。
<制約条件>
- 要点は3~5つに絞って下さい。
- 日本語で要約して下さい。
- Formatの内容以外のことは出力しないでください。
<Format>
```
{{記事全体の要約を100字程度で出力する}}
```
1. *{{要点1見出し}}* ：{{要点1のまとめ}}
2. *{{要点2見出し}}* ：{{要点2のまとめ}}
...
n. *{{要点n見出し}}* ：{{要点nのまとめ}}
<Content>
{entry.title}
{entry.get('content', '')}
"""
            logger.debug(f"summary_prompt: {prompt}")

            completion = client.chat.completions.create(
                model="gpt-4", messages=[{"role": "system", "content": prompt}]
            )
            summaries.append(
                {
                    "title": entry.title,
                    "summary": completion.choices[0].message.content,
                    "link": entry.link,
                }
            )
            mark_news_as_processed(news_id, processed_news)
        except Exception as e:
            logger.error(f"Error summarizing news: {e}")
    return summaries


def send_to_slack(summaries, webhook_url):
    if not webhook_url:
        logger.error("Slack Webhook URL is not set.")
        return

    for summary in summaries:
        try:
            message = {
                "text": f"*<{summary['link']}|{summary['title']}>*\n{summary['summary']}",
                "unfurl_links": True,
            }
            response = requests.post(webhook_url, json=message)
            if response.status_code != 200:
                logger.error(f"Failed to send message to Slack: {response.status_code}")

            logger.debug(f"message: {message}")
        except Exception as e:
            logger.error(f"Error sending to Slack: {e}")


def main():
    # 環境変数のチェック
    slack_webhook_url = os.getenv("SLACK_WEBHOOK_URL")
    if not slack_webhook_url:
        logger.error("Slack Webhook URL is not set.")
        return

    # 複数のRSSフィードからニュースを取得
    rss_urls = [
        "https://qiita.com/popular-items/feed",
        "https://gigazine.net/news/rss_2.0/",
        "https://b.hatena.ne.jp/entrylist/it.rss",
        "https://dev.classmethod.jp/feed/",
        "https://news.microsoft.com/ja-jp/feed/",
        "https://aws.amazon.com/jp/about-aws/whats-new/recent/feed/",
        "https://zenn.dev/feed",
    ]
    feed_entries = get_rss_feeds(rss_urls)

    # ニュースから生成AIに関連するものを抽出
    ai_related_entries = filter_ai_news(feed_entries)

    # 処理済みニュースの読み込み
    processed_news = load_processed_news()

    # 古いニュースを削除
    processed_news = clean_old_news(processed_news)

    # ニュースのサマリを生成
    summaries = summarize_news(ai_related_entries, processed_news)

    # 処理済みニュースの保存
    save_processed_news(processed_news)

    # サマリをSlackに送信
    send_to_slack(summaries, slack_webhook_url)

In [22]:
main()

message: {'text': '*<https://qiita.com/kumukai/items/56c380e337a3699aa8a0?utm_campaign=popular_items&utm_medium=feed&utm_source=popular_items|Power Platform で市民開発するなら、環境ルーティングを設定しよう！>*\n```\nPower Platformで市民開発する際の環境ルーティング設定の方法とその意義について解説します。\n```\n1. *個人用開発環境の作成* ：マネージド環境の機能である環境ルーティングを活用して個人用開発環境を設定する方法を紹介。\n2. *環境ルーティングの意義* ：環境ルーティングを使用することの意義を説明。\n3. *マネージド環境の便利さ* ：マネージド環境の具体的な利便性について説明。\n', 'unfurl_links': True}
message: {'text': '*<https://qiita.com/shuji001/items/7046f8fe95c50f45e512?utm_campaign=popular_items&utm_medium=feed&utm_source=popular_items|レジにお客さまが並んだら教えてくれる！ 画像認識ツールを使えばずっとレジにいなくてもいいんです！>*\n```\nDXの一環として、画像認識ツールを活用し、スーパーのレジ業務を効率化する試みが紹介されています。\n```\n1. *DXによる課題解決* ：スーパーで働く筆者はDXを通じて業務課題の解決を模索している。\n2. *画像認識ツールの活用* ：画像認識ツールを使い、お客さまがレジに並んだ際に通知を受け取るシステムを考えている。\n3. *効率化の狙い* ：ツールにより、常にレジにいなくても対応できるようにし、業務の効率化を図る。\n', 'unfurl_links': True}


In [ ]:
# 複数のRSSフィードからニュースを取得
rss_urls = [
    "https://qiita.com/popular-items/feed",
    "https://gigazine.net/news/rss_2.0/",
    "https://b.hatena.ne.jp/entrylist/it.rss",
    "https://dev.classmethod.jp/feed/",
    "https://news.microsoft.com/ja-jp/feed/",
    "https://aws.amazon.com/jp/about-aws/whats-new/recent/feed/",
    "https://zenn.dev/feed",
]
feed_entries = get_rss_feeds(rss_urls)